In [1]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.1.2/libexec")
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

from pyspark.sql.functions import*
from pyspark.sql.types import*
import time

import os

import numpy as np
import re
import pandas as pd 
import traceback
import sys
from time import sleep
import traceback
import smtplib


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
###WASHINGTON

jdbcUrl="jdbc:db2://67beb010-4100-48d1-a281-7b5af97791aa.bs2ipa7w0uv9tsomi9ig.databases.appdomain.cloud:31815/bludb:sslConnection=true;"
user="a5406f1c"
password="eK1EhADAU7y3W61T"

In [4]:
def writeToSQLWarehouse(df,epochId,tbl):
  print(tbl)
  print(df)
  df.write.format("jdbc") \
  .mode("append") \
  .option("url", jdbcUrl) \
  .option("dbtable", tbl) \
  .option("user", user) \
  .option("password", password) \
  .save()

In [5]:
#ctry=lookupItemID(lookuptablelist,'100')

In [6]:
tbllist=[]
jsonpathlists=[]
colslist=[]

with open('json_column_map.txt') as f:
    contents = f.read()
    contents=contents.strip()
    maplist=contents.split("TABLE_NAME:")
    #print(maplist)
    #maplist = list(filter([], maplist))
    for lmap in maplist:
        lmaplist=lmap.split("\n")
        #lmaplist = list(filter(None, lmaplist))
        tbl=lmaplist.pop(0)
        tbllist.append(tbl)
        collist=[]
        jsonpathlist=[]
        for col_json in lmaplist:
            coljsonarr=col_json.split(",")
            if (len(coljsonarr)==2):
               collist.append(coljsonarr[0])
               jsonpathlist.append(coljsonarr[1])
        colslist.append(collist)
        jsonpathlists.append(jsonpathlist)
            
            
f.close()


tbllist = [x for x in tbllist if x != 'KAFKARADIAL.']
tbllist = [x for x in tbllist if x != '']
colslist = [x for x in colslist if x != []]
jsonpathlists = [x for x in jsonpathlists if x != []]

for tbl, jsonpathlist,collist in zip(tbllist, jsonpathlists, colslist):
    print(tbl)
    print(jsonpathlist)
    print(collist)
    print("********")

KAFKARADIAL.ROM_ORDER_HEADER_STG0
['order.id', 'order.sellerId', 'order.customerInfo.phone', 'order.customerInfo.customerId', 'order.customerOrderId', 'order.type', 'order.orderClosed', 'order.orderSourceCode']
['ORDER_ID', 'SELLER_ID', 'CUSTOMER_PHONE', 'CUSTOMER_ID', 'CUSTOMER_ORDER_ID', 'ORDER_TYPE', 'ORDER_CLOSED_FLAG', 'ORSO_CODE']
********
KAFKARADIAL.ROM_ORDER_LINE_STG0
['order.id', 'order.lineItems.0.sizeCode', 'order.sellerId', 'order.lineItems.0.color', 'order.lineItems.0.isAssociateDelivery', 'order.lineItems.0.size', 'order.lineItems.0.colorCode', 'order.lineItems.0.itemDetail.unitOfMeasure', 'order.lineItems.0.itemDetail.itemId', 'order.lineItems.0.itemDetail.gii', 'order.lineItems.0.lineNo']
['ORDER_ID', 'SIZE_CODE', 'SELLER_ID', 'COLOR', 'IS_ASSOCIATE_DELIVERY', 'SIZE', 'COLOR_CODE', 'ITEM_UOM', 'ITEM_ID', 'ITEM_GII', 'ORDER_LINE_KEY']
********
KAFKARADIAL.ROM_RELATED_ORDERS_STG0
['order.id', 'order.sellerId', 'order.type']
['ORDER_ID', 'SELLER_ID', 'ORDER_TYPE']
*******

In [7]:
'order_lineItems_charges_taxes.jurisdictionLevel',  'order_lineItems_charges_taxes.situs',

('order_lineItems_charges_taxes.jurisdictionLevel',
 'order_lineItems_charges_taxes.situs')

In [8]:
#### Obtaining Fields for Table2 

def explodeDF(df):
    
    elm = "order.lineItems.0.charges.0.taxes.0.jurisdictionLevel"
    df=df.withColumn("order_lineItems",explode("order.lineItems"))
    df.show()
    
    df=df.withColumn("order_lineItems_charges",explode("order_lineItems.charges"))
    df.show()
    
    df=df.withColumn("order_lineItems_charges_taxes",explode("order_lineItems_charges.taxes"))
    df.show()
    
    return df

In [9]:
def checkPathExists(jsonpathlist, collist,df):

    jsonpathwithval=[]
    collistwithval=[]

    for jpath,col in zip(jsonpathlist,collist):
        
       try:
            df.select(jpath)
            jsonpathwithval.append(jpath)
            collistwithval.append(col)
       except Exception as e:
            print(jpath+" "+col )
        
    return jsonpathwithval, collistwithval
        

In [10]:
import builtins
round = getattr(builtins, "round")
from pyspark.sql.functions import lit

def updateDF(sdf,tbl):
    
    sdf.printSchema()
    if tbl=="KAFKARADIAL.ROM_ORDER_LINE_STG0":
       try:
        sdf=sdf.withColumn("IS_ASSOCIATE_DELIVERY", when(col("IS_ASSOCIATE_DELIVERY") == "false",lit("0") )\
        .when(col("IS_ASSOCIATE_DELIVERY") == "true",lit("1")) \
        .otherwise(col("IS_ASSOCIATE_DELIVERY")))
       except Exception as e:
         print(e)
    
    return sdf

In [11]:
def prepareInsertTable1(df,epochID):
    
    table1='KAFKARADIAL.ROM_ORDER_HEADER_STG0'
    jsonpath1 = ['order.id', 'order.sellerId', 'order.customerInfo.phone', 'order.customerInfo.customerId', 
                 'order.customerOrderId', 'order.type', 'order.orderClosed', 'order.orderSourceCode']
    jsonpath1up=[
    
         'order.id',
         'order.sellerId',
         'order.customerInfo.phone',
         'order.customerInfo.customerId',
         'order.customerOrderId',
         'order.type',
         'order.orderClosed',
         'order.orderSourceCode'
    
        ]
    collist1 = ['ORDER_ID', 'SELLER_ID', 'CUSTOMER_PHONE', 'CUSTOMER_ID', 'CUSTOMER_ORDER_ID', 
                'ORDER_TYPE', 'ORDER_CLOSED_FLAG', 'ORSO_CODE']
    jsonpath1_withval,collist1_withval = checkPathExists(jsonpath1up, collist1,df)
    #print(jsonpath1_withval)
   
    
    ###### Selecting the corresponding values for the Table1 from the Dataframe #####
    cdf = df.select(jsonpath1_withval)
    cdf = df.select('order.id', 'order.sellerId', 'order.customerInfo.phone', 'order.customerInfo.customerId', 'order.customerOrderId', 'order.type', 'order.orderClosed', 'order.orderSourceCode')
    cdf.show()
    
    #print(collist1_withval)
    
    ##### Renaming Columns to Match Actual Table
    sdf=cdf.toDF(*collist1_withval)
    sdf.show()
    
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table1)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table1)
    

In [12]:
def prepareInsertTable2(df,epochID):
    
    table2= "KAFKARADIAL.ROM_ORDER_LINE_STG0"
    jsonpath2 = ['order.id', 'order.lineItems.0.sizeCode', 'order.sellerId', 'order.lineItems.0.color', 'order.lineItems.0.isAssociateDelivery', 'order.lineItems.0.size', 'order.lineItems.0.colorCode', 'order.lineItems.0.itemDetail.unitOfMeasure', 'order.lineItems.0.itemDetail.itemId', 'order.lineItems.0.itemDetail.gii', 'order.lineItems.0.lineNo']
    jsonpath2up=[
    
     'order.id',
     'order_lineItems.sizeCode',
     'order.sellerId',
     'order_lineItems.color',
     'order_lineItems.isAssociateDelivery',
     'order_lineItems.size',
     'order_lineItems.colorCode',
     'order_lineItems.itemDetail.unitOfMeasure',
     'order_lineItems.itemDetail.itemId',
     'order_lineItems.itemDetail.gii',
     'order_lineItems.lineNo'
    
    ]
    collist2=['ORDER_ID', 'SIZE_CODE', 'SELLER_ID', 'COLOR', 'IS_ASSOCIATE_DELIVERY', 'SIZE', 
              'COLOR_CODE', 'ITEM_UOM', 'ITEM_ID', 'ITEM_GII', 'ORDER_LINE_KEY']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath2_withval,collist2_withval = checkPathExists(jsonpath2up, collist2, df)
    print(jsonpath2_withval)    
    
    ###### Selecting the corresponding values for the Table1 from the Dataframe #####
    cdf = df.select(jsonpath2_withval) # Same as below
    cdf = df.select('order.id','order_lineItems.sizeCode','order.sellerId','order_lineItems.color','order_lineItems.isAssociateDelivery', 'order_lineItems.size', 'order_lineItems.colorCode', 'order_lineItems.itemDetail.unitOfMeasure', 'order_lineItems.itemDetail.itemId', 'order_lineItems.itemDetail.gii', 'order_lineItems.lineNo' )
    cdf.show()
        
    ### Renaming Columns
    sdf=cdf.toDF(*collist2_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table2)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table2)


In [13]:
def prepareInsertTable3(df,epochID):
    
    table3 = "KAFKARADIAL.ROM_RELATED_ORDERS_STG0"
    jsonpath3 = ['order.id', 'order.sellerId', 'order.type']
    jsonpath3up=[
    
        'order.id','order.sellerId', 'order.type'
    
    ]
    collist3 = ['ORDER_ID', 'SELLER_ID', 'ORDER_TYPE']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath3_withval,collist3_withval = checkPathExists(jsonpath3up, collist3, df)
    print(jsonpath3_withval)
    
    ###### Selecting the corresponding values for the Table1 from the Dataframe #####
    cdf = df.select(jsonpath3_withval)
    cdf = df.select('order.id', 'order.sellerId', 'order.type')
    cdf.show()
        
    ##### Renaming Columns
    sdf=cdf.toDF(*collist3_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table3)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()

    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table3)
    

In [14]:
def prepareInsertTable4(df,epochID):
    
    table4 = "KAFKARADIAL.ROM_ORDER_PAYMENT_STG0"
    jsonpath4 = ['order.id', 'order.paymentMethods.0.displayCreditCardNumber',
                 'order.paymentMethods.0.expiryDate', 'order.sellerId', 
                 'order.paymentMethods.0.tenderType', 'order.paymentMethods.0.code']
    
    jsonpath4up=[
    
         'order.id',
         'order_paymentMethods.displayCreditCardNumber',
         'order_paymentMethods.expiryDate',
         'order.sellerId',
         'order_paymentMethods.tenderType',
         'order_paymentMethods.code'
    
    ]
    collist4 = ['ORDER_ID', 'DISPLAY_ACCT_NO', 'EXPIRY_DATE', 'SELLER_ID', 
                'TENDER_TYPE', 'PAYMENT_METHOD_CODE']

    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath4_withval,collist4_withval = checkPathExists(jsonpath4up, collist4, df) 
    print(jsonpath4_withval)
    
     
    ###### Selecting the corresponding values for the Table1 from the Dataframe #####
    cdf = df.select(jsonpath4_withval)
    cdf = df.select('order.id', 'order.sellerId')
    cdf.show()
        
    ##### Renaming Columns
    sdf=cdf.toDF(*collist4_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table4)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table4)
    

In [15]:
def prepareInsertTable5(df,epochID):
    
    table5 = "KAFKARADIAL.ROM_ORDER_PROMOTION_STG0"
    jsonpath5 = ['order.id', 'order.sellerId']
    jsonpath5up=[
    
        'order.id', 'order.sellerId'
    
    ]
    
    collist5 = ['ORDER_ID', 'SELLER_ID']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath5_withval,collist5_withval = checkPathExists(jsonpath5up, collist5, df)
    print(jsonpath5_withval)
    
    cdf = df.select(jsonpath5_withval)
    cdf = df.select('order.id', 'order.sellerId')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist5_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table5)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table5)
    

In [16]:
def prepareInsertTable6(df,epochID):
    
    table6 = "KAFKARADIAL.ROM_ORDER_REFERENCES_STG0"
    jsonpath6 = ['order.id', 'order.sellerId']
    jsonpath6up=[
    
        'order.id', 'order.sellerId'
    
    ]
    
    collist6 = ['ORDER_ID', 'SELLER_ID']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath6_withval,collist6_withval = checkPathExists(jsonpath6up, collist6, df)
    print(jsonpath6_withval)
    
    cdf = df.select(jsonpath6_withval)
    cdf = df.select('order.id', 'order.sellerId')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist6_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table6)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table6)
    

In [17]:
def prepareInsertTable7(df,epochID):
    
    table7 = "KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0"
    jsonpath7 = ['order.lineItems.0.charges.0.taxes.0.jurisdictionLevel', 'order.id', 'order.lineItems.0.charges.0.taxes.0.situs', 'order.lineItems.0.charges.0.taxes.0.impositionValue', 'order.sellerId',
                 'order.lineItems.0.charges.0.taxes.0.impositionType', 'order.lineItems.0.charges.0.taxes.0.jurisdictionId']
    jsonpath7up=[
    
     'order_lineItems_charges_taxes.jurisdictionLevel',
     'order.id',
     'order_lineItems_charges_taxes.situs',
     'order_lineItems_charges_taxes.impositionValue',
     'order.sellerId',
     'order_lineItems_charges_taxes.impositionType',
     'order_lineItems_charges_taxes.jurisdictionId'
        
    ]
    
    collist7 = ['JURISDICTION_LEVEL', 'ORDER_ID', 'TAX_SITUS', 
                'IMPOSITION_VALUE', 'SELLER_ID', 'IMPOSITION_TYPE', 'JURISDICTION_ID']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath7_withval,collist7_withval = checkPathExists(jsonpath7up, collist7, df)
    print(jsonpath7_withval)
    
    cdf = df.select(jsonpath7_withval)
    cdf = df.select('order_lineItems_charges_taxes.jurisdictionLevel',
     'order.id',
     'order_lineItems_charges_taxes.situs',
     'order_lineItems_charges_taxes.impositionValue',
     'order.sellerId',
     'order_lineItems_charges_taxes.impositionType',
     'order_lineItems_charges_taxes.jurisdictionId')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist7_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table7)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table7)
    

In [18]:
def prepareInsertTable8(df,epochID):
    
    table8 = "KAFKARADIAL.ROM_ORDER_LINE_RELATIONSHIP_STG0"
    jsonpath8 = ['order.id', 'order.sellerId']
    jsonpath8up=[
    
        'order.id', 'order.sellerId'
    
        ]
    collist8 = ['ORDER_ID', 'SELLER_ID']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath8_withval,collist8_withval = checkPathExists(jsonpath8up, collist8, df)
    print(jsonpath8_withval)
    
    cdf = df.select(jsonpath8_withval)
    cdf = df.select('order.id', 'order.sellerId')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist8_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table8)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table8)
    
    

In [19]:
def prepareInsertTable9(df,epochID):
    
    table9 = "KAFKARADIAL.ROM_ORDER_CUSTOMER_INFO_STG0"
    jsonpath9 = ['order.id', 'order.sellerId']
    jsonpath9up=[
    
        'order.id', 'order.sellerId'
    
    ]
    collist9 = ['ORDER_ID', 'SELLER_ID']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath9_withval,collist9_withval = checkPathExists(jsonpath9up, collist9, df)
    print(jsonpath9_withval)
    cdf = df.select(jsonpath9_withval)
    cdf = df.select('order.id', 'order.sellerId')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist9_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table9)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table9)
    
    

In [20]:
def prepareInsertTable10(df,epochID):
    
    table10 = "KAFKARADIAL.ROM_ORDER_LINE_CHARGES_STG0"
    jsonpath10 = ['order.id', 'order.sellerId']
    jsonpath10up=[
    
        'order.id', 'order.sellerId'
    
    ]
    collist10 = ['ORDER_ID', 'SELLER_ID']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath10_withval,collist10_withval = checkPathExists(jsonpath10up, collist10, df)
    print(jsonpath10_withval)
    cdf = df.select(jsonpath10_withval)
    cdf = df.select('order.id', 'order.sellerId')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist10_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table10)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table10)



In [21]:
def prepareInsertTable11(df,epochID):
    
    table11 = "KAFKARADIAL.ROM_ORDER_LINE_STATUS_STG0"
    jsonpath11 = ['order.lineItems.0.sublineItems.0.statusRank ', 'order.id', 'order.sellerId', 'order.lineItems.0.itemDetail.itemId', 'order.type']
    jsonpath11up=[
    
     'order_lineItems_sublineItems.statusRank ',
     'order.id',
     'order.sellerId',
     'order_lineItems.itemDetail.itemId',
     'order.type'
    
    ]
    collist11 = ['STATUS_ID', 'ORDER_ID', 'SELLER_ID', 'ITEM_ID', 'ORDER_TYPE']
    
    ##### Check for Paths Existing in the JSON Msg #########
    jsonpath11_withval,collist11_withval = checkPathExists(jsonpath11up, collist11, df)
    print(jsonpath11_withval)
    cdf = df.select(jsonpath11_withval)
    cdf = df.select(
     'order.id',
     'order.sellerId',
     'order_lineItems.itemDetail.itemId',
     'order.type')
    cdf.show()
        
    ## Renaming the Columns
    sdf=cdf.toDF(*collist11_withval)
    sdf.show()
    
    ##### Update Dataframe ######
    sdf=updateDF(sdf,table11)
    
    ##### Drop Duplicates ######
    print("Dropping Duplicates")
    finaldf=sdf.dropDuplicates()
    finaldf.show()
    
    ##### Inserting to Database #####
    writeToSQLWarehouse(finaldf,epochID,table11)
    

In [22]:
def prepMultiRows(df,epochID):
    df.show()
    l = []
    for row in df.rdd.collect():
       x=row[0]
       dfr = spark.read.option("multiline", "true") \
       .json(spark.sparkContext.parallelize([x]))
       dfr.show()
       l.append(dfr)
    print("***")

    return l

In [23]:
def popTables(df, epochID):
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    #df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "msg")
    dflist=prepMultiRows(df,epochID)
    print("after")
    for jmsg in dflist:
        jmsg.show()
        #print(jsonpathlist)
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            
            ###### Exploding Dataframe #######
            
            jmsg=explodeDF(jmsg)
            
            
            ###### Preparing Dataframe and Inserting to DB #######
            
            prepareInsertTable1(jmsg,epochID)
            prepareInsertTable2(jmsg,epochID)
            prepareInsertTable3(jmsg,epochID)
            prepareInsertTable4(jmsg,epochID)
            prepareInsertTable5(jmsg,epochID)
            prepareInsertTable6(jmsg,epochID)
            prepareInsertTable7(jmsg,epochID)
            prepareInsertTable8(jmsg,epochID)
            prepareInsertTable9(jmsg,epochID)
            prepareInsertTable10(jmsg,epochID)
            prepareInsertTable11(jmsg,epochID)
            
        else:
            print("Corrupt Record:")
            jmsg.show()
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)

In [24]:
####### Reading Input Stream in JSON Format from Kafka ########

try:

 df = spark \
  .readStream \
  .option("multiLine", True).option("mode", "PERMISSIVE") \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test3") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)


In [25]:
####### Writing Steams to DB #######

query1 = df \
    .writeStream \
    .outputMode("append") \
    .option("partition.assignment.strategy", "range")\
    .foreachBatch(popTables)\
    .start()

query1.awaitTermination(10)

<bound method RDD.isEmpty of MapPartitionsRDD[9] at javaToPython at NativeMethodAccessorImpl.java:0>


False

before
+--------------------+
|               value|
+--------------------+
|{  "topicName" : ...|
+--------------------+

+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

***
after
+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

****
+----------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|
+----------------+--------------------+--------------------+--------------------+
|   

+-----------+---------+---------+-----+---------------------+------+----------+--------+-------------+--------+--------------+
|   ORDER_ID|SIZE_CODE|SELLER_ID|COLOR|IS_ASSOCIATE_DELIVERY|  SIZE|COLOR_CODE|ITEM_UOM|      ITEM_ID|ITEM_GII|ORDER_LINE_KEY|
+-----------+---------+---------+-----+---------------------+------+----------+--------+-------------+--------+--------------+
|30002217646|       MD|     浣熊|Black|                    0|Medium|       001|    EACH|75-MH06-MD-BK|32306995|             1|
+-----------+---------+---------+-----+---------------------+------+----------+--------+-------------+--------+--------------+

KAFKARADIAL.ROM_ORDER_LINE_STG0
DataFrame[ORDER_ID: string, SIZE_CODE: string, SELLER_ID: string, COLOR: string, IS_ASSOCIATE_DELIVERY: string, SIZE: string, COLOR_CODE: string, ITEM_UOM: string, ITEM_ID: string, ITEM_GII: string, ORDER_LINE_KEY: string]
['order.id', 'order.sellerId', 'order.type']
+-----------+--------+-----+
|         id|sellerId| type|
+-------

+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
+-----------+---------+

root
 |-- ORDER_ID: string (nullable = true)
 |-- SELLER_ID: string (nullable = true)

Dropping Duplicates
+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
+-----------+---------+

KAFKARADIAL.ROM_ORDER_CUSTOMER_INFO_STG0
DataFrame[ORDER_ID: string, SELLER_ID: string]
['order.id', 'order.sellerId']
+-----------+--------+
|         id|sellerId|
+-----------+--------+
|30002217646|    浣熊|
|30002217646|    浣熊|
|30002217646|    浣熊|
|30002217646|    浣熊|
+-----------+--------+

+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
|30002217646|     浣熊|
+-----------+---------+

root
 |-- ORDER_ID: string (nullable = true)
 |-- SELLER_ID: string (nullable = true)

Dropping Duplicates
+

In [ ]:
query1.stop()

In [ ]:
query1.status